### Connect to Drive

In [ ]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive/My Drive/Progetto AN2DL/5_ImageAugmentation

### Import libraries

In [ ]:
# Hide warnings
import os

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=Warning)

import numpy as np

import logging

In [ ]:
# Import tensorflow
import tensorflow as tf
from tensorflow import keras as tfk
from tensorflow.keras import layers as tfkl
from tensorflow.keras.preprocessing.image import ImageDataGenerator
tf.autograph.set_verbosity(0)
tf.get_logger().setLevel(logging.ERROR)
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)
print(tf.__version__)

2.14.0


In [ ]:
# Import other libraries
import cv2
from tensorflow.keras.applications.mobilenet import preprocess_input
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, KFold
import pickle

### Load and process the dataset

In [ ]:
# Load data
data = np.load('../clean_data.npz', allow_pickle=True)

# Save images in a new list (image pixel values are float in [0, 1])
images = data['data']

# Save labels in a new list
labels = data['labels']

# Print data shape
print(images.shape, labels.shape)

(5004, 96, 96, 3) (5004,)


In [ ]:
# Convert labels from string to integer
num_labels = []
for label in labels:
  if label == "healthy":
    num_labels.append(0)
  else:
    num_labels.append(1)

# Convert labels in the one-hot encoding format
num_labels = np.array(num_labels)
labels = tfk.utils.to_categorical(num_labels, 2)

# Split data into train and val sets
X_train, X_val, y_train, y_val = train_test_split(
    images,
    labels,
    test_size=0.1,
    stratify=labels
)

# Print shapes of the datasets
print(f"X_train shape: {X_train.shape}, y_train shape: {y_train.shape}")
print(f"X_val shape: {X_val.shape}, y_val shape: {y_val.shape}")

X_train shape: (4503, 96, 96, 3), y_train shape: (4503, 2)
X_val shape: (501, 96, 96, 3), y_val shape: (501, 2)


### ConvNeXtLarge

In [ ]:
# Import Feature Extractor with specified settings
feature_extractor = tfk.applications.ConvNeXtLarge(
    input_shape=(96, 96, 3),
    include_top=False,
    weights="imagenet",
    pooling='avg',
    include_preprocessing=True,
)

### Model

In [ ]:
# Define network parameters/callbacks
dropout_rate = 1/10
learning_rate = 1e-3

lr_scheduler = tfk.callbacks.ReduceLROnPlateau(
    monitor='val_accuracy',
    patience=5,
    factor=0.999,
    mode='max',
    min_lr=1e-5
)

early_stopping = tfk.callbacks.EarlyStopping(
    monitor='val_accuracy',
    mode='max',
    patience=20,
    restore_best_weights=True
)

In [ ]:
def build_model(feature_extractor, dropout_rate, learning_rate=0.001):
  # Use the supernet as feature extractor, i.e. freeze all its weigths
  feature_extractor.trainable = False

  # Create an input layer with shape (96, 96, 3)
  inputs = tfk.Input(shape=(96, 96, 3), name='Input')

  # Connect FeatureExtractor to the input
  x = feature_extractor(inputs)

  # Hidden layers
  x = tfkl.Dense(units=256, name='HiddenDense1')(x)
  x = tfkl.Activation('relu', name='HiddenActivation1')(x)
  dropout = tfkl.Dropout(dropout_rate)(x)
  x = tfkl.Dense(units=256, name='HiddenDense2')(dropout)
  x = tfkl.Activation('relu', name='HiddenActivation2')(x)
  dropout = tfkl.Dropout(dropout_rate)(x)

  # Add a Dense layer with 2 unit and softmax activation as the classifier
  outputs = tfkl.Dense(2, activation='softmax')(dropout)

  # Create a Model connecting input and output
  model = tfk.Model(inputs=inputs, outputs=outputs, name='model')

  # Compile the model with Categorical Cross-Entropy loss and Adam optimizer
  model.compile(loss=tfk.losses.CategoricalCrossentropy(),
                optimizer=tfk.optimizers.Adam(learning_rate),
                metrics=['accuracy'])

  # Return the model
  return model

### FLIP + SHIFT + ROT

In [ ]:
model = build_model(feature_extractor, dropout_rate, learning_rate)
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Input (InputLayer)          [(None, 96, 96, 3)]       0         
                                                                 
 convnext_large (Functional  (None, 1536)              196230336 
 )                                                               
                                                                 
 HiddenDense1 (Dense)        (None, 256)               393472    
                                                                 
 HiddenActivation1 (Activat  (None, 256)               0         
 ion)                                                            
                                                                 
 dropout_6 (Dropout)         (None, 256)               0         
                                                                 
 HiddenDense2 (Dense)        (None, 256)               65792 

In [ ]:
train_datagen = ImageDataGenerator(
        horizontal_flip=True,
        vertical_flip = True,
        width_shift_range = 0.1,
        height_shift_range = 0.1,
        rotation_range=90.,
        fill_mode='reflect')

train_datagen.fit(X_train)

In [ ]:
history_flip_shift_rot = model.fit_generator(train_datagen.flow(X_train*255, y_train,
                              batch_size=16),
                              epochs=200,
                              steps_per_epoch=len(X_train)/16,
                              validation_data = (X_val*255, y_val),
                              callbacks = [early_stopping, lr_scheduler]
                              ).history

Epoch 1/200
281/281 [==============================] - 71s 177ms/step - loss: 0.4902 - accuracy: 0.7688 - val_loss: 0.3671 - val_accuracy: 0.8343 - lr: 0.0010
Epoch 2/200
281/281 [==============================] - 45s 158ms/step - loss: 0.3805 - accuracy: 0.8279 - val_loss: 0.3488 - val_accuracy: 0.8523 - lr: 0.0010
Epoch 3/200
281/281 [==============================] - 46s 163ms/step - loss: 0.3633 - accuracy: 0.8383 - val_loss: 0.5313 - val_accuracy: 0.8184 - lr: 0.0010
Epoch 4/200
281/281 [==============================] - 46s 162ms/step - loss: 0.3227 - accuracy: 0.8525 - val_loss: 0.3596 - val_accuracy: 0.8303 - lr: 0.0010
Epoch 5/200
281/281 [==============================] - 46s 162ms/step - loss: 0.3031 - accuracy: 0.8708 - val_loss: 0.3216 - val_accuracy: 0.8643 - lr: 0.0010
Epoch 6/200
281/281 [==============================] - 47s 166ms/step - loss: 0.3044 - accuracy: 0.8696 - val_loss: 0.2944 - val_accuracy: 0.8762 - lr: 0.0010
Epoch 7/200
281/281 [=========================

In [ ]:
# Plot the training histories
plt.figure(figsize=(15,5))
plt.plot(history_flip_shift_rot['loss'], alpha=.3, color='#4D61E2', linestyle='--')
plt.plot(history_flip_shift_rot['val_loss'], label='Flip+Shift+Rotation', alpha=.8, color='#4D61E2')
plt.legend(loc='upper left')
plt.title('Categorical Crossentropy')
plt.grid(alpha=.3)

plt.figure(figsize=(15,5))
plt.plot(history_flip_shift_rot['accuracy'], alpha=.3, color='#4D61E2', linestyle='--')
plt.plot(history_flip_shift_rot['val_accuracy'], label='Flip+Shift+Rotation', alpha=.8, color='#4D61E2')
plt.legend(loc='upper left')
plt.title('Accuracy')
plt.grid(alpha=.3)

plt.show()

In [ ]:
# Save the model
model.save('Augmentation_Flip_Shift_Rot')
with open('Augmentation_Flip_Shift_Rot/history.pkl', 'wb') as f:
  pickle.dump(history_flip_shift_rot, f)